In [11]:
import numpy as np
import pandas as pd
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
from imblearn.over_sampling import ADASYN
import torch
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [12]:
model_name = "sentence-transformers/LaBSE"
tokenizer1 = AutoTokenizer.from_pretrained(model_name)
model1 = AutoModel.from_pretrained(model_name)

model1.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(501153, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=Fals

In [5]:
data  = pd.read_csv('./Data/cleaned_PS_train.csv')
le = LabelEncoder()
data['label'] = le.fit_transform(data['label'])

val = pd.read_csv('./Data/merged_test.csv')
val['label'] = le.transform(val['label'])

print("Class mapping (Original label -> Encoded integer):")
for i, class_name in enumerate(le.classes_):
    print(f"{class_name} -> {i}")

# Print the classes array
print("\nOriginal classes:")
print(le.classes_)
 

Class mapping (Original label -> Encoded integer):
Negative -> 0
Neutral -> 1
None of the above -> 2
Opinionated -> 3
Positive -> 4
Sarcastic -> 5
Substantiated -> 6

Original classes:
['Negative' 'Neutral' 'None of the above' 'Opinionated' 'Positive'
 'Sarcastic' 'Substantiated']


In [13]:
model_name = "ai4bharat/indic-bert"
tokenizer2 = AutoTokenizer.from_pretrained(model_name)
model2 = AutoModel.from_pretrained(model_name)

model2.to(device)

AlbertModel(
  (embeddings): AlbertEmbeddings(
    (word_embeddings): Embedding(200000, 128, padding_idx=0)
    (position_embeddings): Embedding(512, 128)
    (token_type_embeddings): Embedding(2, 128)
    (LayerNorm): LayerNorm((128,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0, inplace=False)
  )
  (encoder): AlbertTransformer(
    (embedding_hidden_mapping_in): Linear(in_features=128, out_features=768, bias=True)
    (albert_layer_groups): ModuleList(
      (0): AlbertLayerGroup(
        (albert_layers): ModuleList(
          (0): AlbertLayer(
            (full_layer_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (attention): AlbertSdpaAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (attention_dropout): Dropout(p=0, inplace=False)
 

Error during conversion: ChunkedEncodingError(ProtocolError('Response ended prematurely'))


In [14]:
def extract_embeddings(model, tokenizer, texts, device = "cuda" if torch.cuda.is_available() else "cpu", max_length=256):
    model.eval()
    embeddings = []
    with torch.no_grad():
        for text in tqdm(texts, desc="Extracting Embeddings"):
            encoding = tokenizer(
                text,
                add_special_tokens=True,
                max_length=max_length,
                padding='max_length',
                truncation=True,
                return_attention_mask=True,
                return_tensors='pt'
            ).to(device)

            # Extract embeddings
            outputs = model(**encoding, output_hidden_states=True)
            hidden_states = outputs.hidden_states[-1]  # Last hidden layer
            embedding = hidden_states.mean(dim=1).squeeze(0)  # Mean pooling
            embeddings.append(embedding.cpu().numpy())

    return torch.tensor(embeddings)


In [15]:
data.shape, val.shape

((4896, 2), (544, 3))

In [16]:
print("Extracting embeddings for training data...")

Extracting embeddings for training data...


In [18]:
X_embeddings1 = extract_embeddings(model1,tokenizer1,data["content"])
X_embeddings2 = extract_embeddings(model2,tokenizer2,data["content"])

val_embeddings1 = extract_embeddings(model1,tokenizer1,val["content"])
val_embeddings2 = extract_embeddings(model2,tokenizer2,val["content"])

Extracting Embeddings: 100%|██████████| 4896/4896 [01:40<00:00, 48.55it/s]
C:\Users\nithi\AppData\Local\Temp\ipykernel_27272\562998904.py:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_new.cpp:281.)
  return torch.tensor(embeddings)
Extracting Embeddings: 100%|██████████| 544/544 [00:08<00:00, 63.07it/s]


In [19]:
print(X_embeddings1.shape, X_embeddings2.shape, val_embeddings1.shape, val_embeddings2.shape)
X = np.concatenate((X_embeddings1, X_embeddings2), axis=1)
val_X = np.concatenate((val_embeddings1, val_embeddings2), axis=1)
y = data['label'].values
y_val = val['label'].values
print(X.shape, y.shape, val_X.shape, y_val.shape)

torch.Size([4896, 768]) torch.Size([4896, 768]) torch.Size([544, 768]) torch.Size([544, 768])
(4896, 1536) (4896,) (544, 1536) (544,)


In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer


tfidf_vectorizer = TfidfVectorizer()
X_tfidf = tfidf_vectorizer.fit_transform(data["content"])

# Create TF-IDF vectors for validation data using the same vocabulary
val_tfidf = tfidf_vectorizer.transform(val["content"])

# Convert to dense arrays for concatenation with embeddings
X_tfidf_dense = X_tfidf.toarray()
val_tfidf_dense = val_tfidf.toarray()

print(f"TF-IDF features shape: {X_tfidf_dense.shape}, {val_tfidf_dense.shape}")

# Create combined feature vectors with embeddings and TF-IDF
X_combined = np.hstack((X, X_tfidf_dense))
val_X_combined = np.hstack((val_X, val_tfidf_dense))

print(f"Combined features shape: {X_combined.shape}, {val_X_combined.shape}")

TF-IDF features shape: (4896, 2540), (544, 2540)
Combined features shape: (4896, 4076), (544, 4076)


In [29]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score

In [30]:
print("Training XGBoost model...")
xgb_model = XGBClassifier(
    n_estimators=200,
    learning_rate=0.1,
    use_label_encoder=False,
    eval_metric='mlogloss'
)
xgb_model.fit(X_combined, y)

# Train LightGBM model
print("Training LightGBM model...")
lgb_model = lgb.LGBMClassifier(
    n_estimators=200,
    learning_rate=0.1,
)
lgb_model.fit(X_combined, y)

# Evaluate models on validation set
xgb_pred = xgb_model.predict(val_X_combined)
lgb_pred = lgb_model.predict(val_X_combined)

# Print results
print("\nXGBoost results:")
print(f"Accuracy: {accuracy_score(y_val, xgb_pred):.4f}")
print(classification_report(y_val, xgb_pred))

print("\nLightGBM results:")
print(f"Accuracy: {accuracy_score(y_val, lgb_pred):.4f}")
print(classification_report(y_val, lgb_pred))

# Perform cross-validation on training data
print("\nPerforming 5-fold cross-validation:")
xgb_cv = cross_val_score(xgb_model, X_combined, y, cv=5, scoring='accuracy')
lgb_cv = cross_val_score(lgb_model, X_combined, y, cv=5, scoring='accuracy')

print(f"XGBoost CV accuracy: {xgb_cv.mean():.4f} ± {xgb_cv.std():.4f}")
print(f"LightGBM CV accuracy: {lgb_cv.mean():.4f} ± {lgb_cv.std():.4f}")

Training XGBoost model...


c:\Users\nithi\anaconda3\envs\College\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:08:21] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Training LightGBM model...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.076068 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 406050
[LightGBM] [Info] Number of data points in the train set: 4896, number of used features: 1923
[LightGBM] [Info] Start training from score -2.371490
[LightGBM] [Info] Start training from score -1.915535
[LightGBM] [Info] Start training from score -3.243900
[LightGBM] [Info] Start training from score -1.173663
[LightGBM] [Info] Start training from score -2.028475
[LightGBM] [Info] Start training from score -1.688239
[LightGBM] [Info] Start training from score -2.356289
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[Li

c:\Users\nithi\anaconda3\envs\College\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



XGBoost results:
Accuracy: 0.3934
              precision    recall  f1-score   support

           0       0.12      0.02      0.04        46
           1       0.26      0.11      0.16        70
           2       0.96      0.92      0.94        25
           3       0.39      0.78      0.52       171
           4       0.32      0.15      0.20        75
           5       0.41      0.33      0.36       106
           6       0.14      0.04      0.06        51

    accuracy                           0.39       544
   macro avg       0.37      0.34      0.33       544
weighted avg       0.35      0.39      0.33       544


LightGBM results:
Accuracy: 0.4007
              precision    recall  f1-score   support

           0       0.20      0.02      0.04        46
           1       0.28      0.16      0.20        70
           2       0.92      0.92      0.92        25
           3       0.38      0.76      0.51       171
           4       0.45      0.19      0.26        75
       

...

c:\Users\nithi\anaconda3\envs\College\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.059306 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 403006
[LightGBM] [Info] Number of data points in the train set: 3917, number of used features: 1865
[LightGBM] [Info] Start training from score -2.373184
[LightGBM] [Info] Start training from score -1.915239
[LightGBM] [Info] Start training from score -3.242643
[LightGBM] [Info] Start training from score -1.173880
[LightGBM] [Info] Start training from score -2.026975
[LightGBM] [Info] Start training from score -1.688290
[LightGBM] [Info] Start training from score -2.356879
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

c:\Users\nithi\anaconda3\envs\College\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.076716 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 403244
[LightGBM] [Info] Number of data points in the train set: 3917, number of used features: 1870
[LightGBM] [Info] Start training from score -2.373184
[LightGBM] [Info] Start training from score -1.915239
[LightGBM] [Info] Start training from score -3.242643
[LightGBM] [Info] Start training from score -1.173054
[LightGBM] [Info] Start training from score -2.028914
[LightGBM] [Info] Start training from score -1.688290
[LightGBM] [Info] Start training from score -2.356879
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furthe

c:\Users\nithi\anaconda3\envs\College\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(


In [24]:
import torch
import torch.nn as nn

class ResidualAttentionBlock(nn.Module):
    def __init__(self, embed_dim, num_heads):
        super(ResidualAttentionBlock, self).__init__()
        self.attn = nn.MultiheadAttention(embed_dim=embed_dim, num_heads=num_heads, batch_first=True)
        self.norm1 = nn.LayerNorm(embed_dim)

        self.ffn = nn.Sequential(
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(embed_dim, embed_dim),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        self.norm2 = nn.LayerNorm(embed_dim)

    def forward(self, x):
        attn_out, _ = self.attn(x, x, x)
        x = self.norm1(x + attn_out)

        ffn_out = self.ffn(x)
        x = self.norm2(x + ffn_out)
        return x

class AttentionClassifier(nn.Module):
    def __init__(self, input_dim=4076, embed_dim=2048, num_heads=16, num_classes=7, num_blocks=8):
        super(AttentionClassifier, self).__init__()

        self.input_proj = nn.Linear(input_dim, embed_dim)
        self.blocks = nn.Sequential(
            *[ResidualAttentionBlock(embed_dim, num_heads) for _ in range(num_blocks)]
        )

        self.pool = nn.AdaptiveAvgPool1d(1)

        self.classifier = nn.Sequential(
            nn.Linear(embed_dim, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.input_proj(x)
        x = x.unsqueeze(1)    

        x = self.blocks(x)             

        x = x.transpose(1, 2)          
        x = self.pool(x).squeeze(-1)   

        return self.classifier(x)


In [28]:
X_train = torch.tensor(X_combined, dtype=torch.float32)
y_train = torch.tensor(y, dtype=torch.long)
X_val = torch.tensor(val_X_combined, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

class_counts = torch.bincount(y_train)
class_weights = 1.0 / class_counts.float()
class_weights = class_weights / class_weights.sum()  # Normalize (optional)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AttentionClassifier().to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights.to(device))
optimizer = optim.Adam(model.parameters(), lr=1e-5)


C:\Users\nithi\AppData\Local\Temp\ipykernel_27272\2847669863.py:4: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_val = torch.tensor(y_val, dtype=torch.long)


X_train shape: torch.Size([4896, 4076]), y_train shape: torch.Size([4896])
X_val shape: torch.Size([544, 4076]), y_val shape: torch.Size([544])


In [29]:
# class FocalLoss(nn.Module):
#     def __init__(self, gamma=2.0, weight=None):
#         super().__init__()
#         self.gamma = gamma
#         self.ce = nn.CrossEntropyLoss(weight=weight)

#     def forward(self, input, target):
#         logp = self.ce(input, target)
#         p = torch.exp(-logp)
#         loss = (1 - p) ** self.gamma * logp
#         return loss.mean(
    
# class_counts = torch.bincount(y_train)
# class_weights = 1.0 / class_counts.float()
# class_weights = class_weights / class_weights.sum()  # Normalize (optional)
# criterion = FocalLoss(gamma=2.0, weight=class_weights.to(device))

In [30]:
from sklearn.metrics import f1_score
num_epochs = 100

for epoch in range(1, num_epochs + 1):
    model.train()
    total_loss = 0
    train_loader_tqdm = tqdm(train_loader, desc=f"Epoch {epoch:02d}")

    for x_batch, y_batch in train_loader_tqdm:
        x_batch, y_batch = x_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        logits = model(x_batch)
        loss = criterion(logits, y_batch)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        train_loader_tqdm.set_postfix(loss=loss.item())

    # Validation
    model.eval()
    correct, total = 0, 0
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for x_batch, y_batch in val_loader:
            x_batch, y_batch = x_batch.to(device), y_batch.to(device)
            logits = model(x_batch)
            preds = torch.argmax(logits, dim=1)

            all_preds.extend(preds.cpu().tolist())
            all_labels.extend(y_batch.cpu().tolist())

            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    acc = correct / total * 100
    macro_f1 = f1_score(all_labels, all_preds, average='macro')
    weighted_f1 = f1_score(all_labels, all_preds, average='weighted')
    micro_f1 = f1_score(all_labels, all_preds, average='micro')
    print(f"Epoch {epoch:02d} | Train Loss: {total_loss:.4f} | Val Acc: {acc:.2f}% | Macro F1: {macro_f1:.4f} | Weighted F1: {weighted_f1:.4f} | Micro F1: {micro_f1:.4f}")

print("\n📊 Classification Report on Validation Set:")
print(classification_report(all_labels, all_preds, digits=4))

Epoch 01: 100%|██████████| 153/153 [00:47<00:00,  3.20it/s, loss=1.82]


Epoch 01 | Train Loss: 270.2272 | Val Acc: 14.34% | Macro F1: 0.1692 | Weighted F1: 0.0747 | Micro F1: 0.1434


Epoch 02: 100%|██████████| 153/153 [00:45<00:00,  3.33it/s, loss=1.45]


Epoch 02 | Train Loss: 253.3594 | Val Acc: 30.88% | Macro F1: 0.2457 | Weighted F1: 0.2605 | Micro F1: 0.3088


Epoch 03: 100%|██████████| 153/153 [00:45<00:00,  3.36it/s, loss=1.62]


...

Epoch 98 | Train Loss: 24.8375 | Val Acc: 32.90% | Macro F1: 0.3530 | Weighted F1: 0.3335 | Micro F1: 0.3290


Epoch 99: 100%|██████████| 153/153 [00:47<00:00,  3.25it/s, loss=0.177] 


Epoch 99 | Train Loss: 26.0676 | Val Acc: 31.43% | Macro F1: 0.3296 | Weighted F1: 0.3179 | Micro F1: 0.3143


Epoch 100: 100%|██████████| 153/153 [00:46<00:00,  3.30it/s, loss=0.13]  


Epoch 100 | Train Loss: 34.1714 | Val Acc: 35.66% | Macro F1: 0.3596 | Weighted F1: 0.3504 | Micro F1: 0.3566

📊 Classification Report on Validation Set:
              precision    recall  f1-score   support

           0     0.0784    0.0870    0.0825        46
           1     0.2609    0.2571    0.2590        70
           2     0.7931    0.9200    0.8519        25
           3     0.4241    0.4737    0.4475       171
           4     0.3556    0.2133    0.2667        75
           5     0.3361    0.3868    0.3596       106
           6     0.2973    0.2157    0.2500        51

    accuracy                         0.3566       544
   macro avg     0.3636    0.3648    0.3596       544
weighted avg     0.3523    0.3566    0.3504       544

